In [ ]:
# import necessary libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re

# change settings to display full string/text in each column
pd.options.display.max_colwidth = 100
np.random.seed(42)

# input space for directory data will be held in

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Step 0. Import data and initial descriptives

- Find # of tweets, RTs, and duplicates
- Parse account_id from Source file and add as a new column
- Drop duplicate text tweets


In [ ]:
# import the merged dataset with all tweets
data = pd.read_csv('drive/MyDrive/F24 Research/data.csv', low_memory=False)

# find total # of tweets
len(data)

375866

In [ ]:
# find # of RTs
RTs = [i for i in data['text'] if (len(i) > 1 and i[:3] == "RT ")]
len(RTs)

162419

In [ ]:
# add column for account ids to new dataframe
# KEY: data_with_ids = all data (including duplicates) with added account_id column
data_with_ids = data.copy()
data_with_ids['account_id'] = data["Source_File"].apply(
    lambda x: re.search(r"tweets_(\d+).csv", x)[1])
# sample random row -- testing that account_ids are matching with the ids found in Source_File
display(data_with_ids[['Source_File', 'account_id']].sample())

# dropping tweets from accounts that are primarily non-English
account_ids_to_drop = [3067215570, 1441091024168783881, 3909684433]
data_with_ids = data_with_ids[~data_with_ids['account_id'].isin(
    account_ids_to_drop)]

,Source_File,account_id
88095,/Users/mvarelaor/Downloads/twitter_mica_jun/output_jun/tweets_1312896813591076864.csv,1312896813591076864


In [ ]:
# create new dataframe with all duplicate tweets
# KEY: dupes = all duplicated tweets with account_id column
dupes = data_with_ids[data_with_ids['text'].duplicated() == True]
# dupes = data_with_ids[data_with_ids[['text', 'account_id']].duplicated() == True]
print(f"{len(dupes)} tweets are duplicates")

13689 tweets are duplicates


In [ ]:
# create new dataset with no duplicates
# KEY: data_no_dupes = original dataset + added account_id - duplicate tweet rows
data_no_dupes = data_with_ids.drop_duplicates(subset='text').copy()
len(data_no_dupes)

362177

# Step 1: Remove RTs


In [ ]:
# remove RTs
mask = data_no_dupes['text'].str[:3] == 'RT '
data_no_dupes = data_no_dupes[~mask]
len(data_no_dupes)
# 1. 362177 -> 211214

211214

# Step 2: Thread Detection

- Twitter "threads" are defined as tweets which are posted by the same account within 1 minute of each other (due to the character limit)
- Combine tweets that are part of threads into one tweet row


In [ ]:
# group the tweets by account id and the time they were created
data_no_dupes.sort_values(by=['account_id', 'created_at'], inplace=True)

# convert 'created_at' to datetime format
data_no_dupes['created_at'] = pd.to_datetime(
    data_no_dupes['created_at'], format='%Y-%m-%dT%H:%M:%SZ', errors='coerce')
data_no_dupes = data_no_dupes.dropna(subset=['created_at'])


# calculate the time difference between consecutive tweets for each account
data_no_dupes['time_diff'] = data_no_dupes.groupby('account_id')[
    'created_at'].diff()

display(data_no_dupes.sample())
len(data_no_dupes)

,...1,text,edit_history_tweet_ids,created_at,id,retweet_count,reply_count,like_count,quote_count,bookmark_count,impression_count,Source_File,withheld,account_id,time_diff
96744,598,Karen Hylton was unjustly arrested after 2 police officers we’re found guilty for being compli...,['1605926873657053184'],2022-12-22 14:02:58,1.605927e+18,30,0,62,2,1,7626,/Users/mvarelaor/Downloads/twitter_mica_jun/output_jun/tweets_1367696320018972675.csv,NaN,1367696320018972675,0 days 11:09:41


211214

In [ ]:
# identify tweet chains within 1 minute
data_no_dupes['chain_id'] = (data_no_dupes['time_diff'] > '00:01:00').cumsum()

In [ ]:
# detecting and storing threads based on their chain id and the account id which posted the tweets
# storing the thread tweets in new variable and removing them from the original dataset
data_no_dupes['in_thread'] = data_no_dupes.duplicated(
    subset=['account_id', 'chain_id'], keep=False)
thread_tweets = data_no_dupes[data_no_dupes['in_thread']]
data_no_dupes = data_no_dupes[~data_no_dupes['in_thread']]

In [ ]:
# length of dataset with no threads
len(data_no_dupes)

161722

In [ ]:
# thread tweet cleaning - combining tweets belonging to the same "thread" together
# group by 'account_id' and 'chain_id' and aggregate tweets and columns as lists for thread tweets
result_df_thread_tweets = thread_tweets.groupby(['account_id', 'chain_id']).agg({
    'created_at': 'first',
    'text': 'first',
    'edit_history_tweet_ids': 'first',
    'id': 'first',
    'retweet_count': 'first',
    'reply_count': 'first',
    'like_count': 'first',
    'quote_count': 'first',
    'bookmark_count': 'first',
    'impression_count': 'first',
    'Source_File': 'first',
    'withheld': 'first',
    'in_thread': 'first',
    'time_diff': 'first',
}).reset_index()

# length of combined threads dataset
len(result_df_thread_tweets)

16311

In [ ]:
# adding thread tweets to the original tweets
combined_data = pd.concat([data_no_dupes, result_df_thread_tweets])

# length of combined dataset
len(combined_data)

178033

# Step 3: Extracting keywords + exclusions

- if 'voting rights act' is the only keyword identified in a tweet, remove it if it is not accompanied with the terms '1965', '1970', '1975', 'anniversary', 'years ago'
- given a set of exclusion terms (terms which contain a keyword but is irrelevant to history, i.e., Emmett Till Foundation), remove tweets if they only contain an exclusion term and nothing else


In [ ]:
# read collected keywords to parse from dataset
keyword_df = pd.read_csv(
    'drive/MyDrive/F24 Research/data/historical_keywords_exclusions.csv', encoding='ISO-8859-1')
keywords = keyword_df['Keywords - ALL'].dropna().tolist()
len(keywords)

981

In [ ]:
data = combined_data.copy()

pattern = re.compile(r'\b(' + '|'.join(re.escape(keyword)
                     for keyword in keywords) + r')\b', re.IGNORECASE)

data['identified_keywords'] = data['text'].str.lower(
).str.strip().str.findall(pattern).apply(lambda x: list(set(x)))

data[[bool(x) for x in data['identified_keywords']]
     ][['text', 'identified_keywords']]

,text,identified_keywords
2144,"As we mourn the loss of Rep. John Lewis, we must also recommit ourselves to advancing his legacy...",[john lewis]
2093,"This false narrative around voter fraud isn't anything new. In fact, it's a voter suppression ta...",[civil war]
1671,"52 years after his murder, the words, &amp; wisdom of Dr. Martin Luther King Jr. continue to lig...","[martin luther king, dr. king]"
1640,Say it loud: I’m BLACK and I’m PROUD! For this Black History/Black Futures Month edition of #Tal...,[black history]
1479,"For generations, racial terror has been used as a weapon to keep communities living in fear &amp...",[racial terror]
...,...,...
16016,"As civil rights leader, Bayard Rustin said “We need, in every community, a group of angelic trou...","[civil rights leader, bayard rustin]"
16018,"Profound author, scholar, social activist, and feminist bell hooks passed away today at the age ...",[bell hooks]
16023,#TodayinBlackHistory ✊🏾 the Student Nonviolent Coordinating Committee (SNCC) sent four voluntee...,"[charles sherrod, diane nash, charles jones]"
16030,"#TodayinBlackHistory ✊🏾 In 1955, Claudette Colvin, a 15-year-old from Montgomery, Alabama refuse...",[claudette colvin]


In [ ]:
print(pd.Series([x for x in data['identified_keywords'] if len(x)][:10]))

0                         [john lewis]
1                          [civil war]
2       [martin luther king, dr. king]
3                      [black history]
4                      [racial terror]
5                        [today marks]
6                       [freedom ride]
7                [juneteenth, slavery]
8      [john lewis, voting rights act]
9    [niagara movement, 116 years ago]
dtype: object


# Step 4: More cleaning - links, punctuation, stop words

- remove links and punctuation

- remove custom stop words


In [ ]:
data_filtered = data.copy()
# removes links and punctuation


def remove_punc(data):
    # remove links
    clean = re.sub(r'http\S+', '', data)
    # remove all numbers unless they are four digits long
    # clean = re.sub(r'\b(?!\d{4}\b)\d+\b', '', clean)
    # remove punctuation
    # clean =  re.sub(r"[^\w\s#'@]", '', clean)
    clean = re.sub(r"[^\w\s]", '', clean)
    # replace non-breaking space with regular spaces
    clean = clean.replace(u'\xa0', u' ')
    # make all lowercase
    clean = clean.lower()
    return clean


# reset index of dataset
data_filtered = data_filtered.reset_index(drop=True)
data_filtered['text_cleaned'] = data_filtered['text'].apply(remove_punc)

# custom stop words list
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
              'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself',
              'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself',
              'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves',
              'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are',
              'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing',
              'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for',
              'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to',
              'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once',
              'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most',
              'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very',
              's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y',
              'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't",
              'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn',
              "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't",
              'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

# include version of the words without ' character
stop_words.extend(list(map(lambda x: x.replace('\'', ''), stop_words)))

stop_words_set = set(stop_words)

# vectorized string operations to remove stop words
data_filtered['text_cleaned'] = data_filtered['text_cleaned'].str.split().apply(
    lambda x: ' '.join([word for word in x if word.lower() not in stop_words_set]))

display(data_filtered[['text', 'text_cleaned']].sample(5))

,text,text_cleaned
52546,Teach In: Money-Handling and Taxes for Mutual Aid Groups https://t.co/6LpLGe0RU5,teach moneyhandling taxes mutual aid groups
74577,"This #WomensHistoryMonth, we are celebrating and uplifting unsung women of color.\n\nBarbara Smi...",womenshistorymonth celebrating uplifting unsung women color barbara smith one many trailblazers ...
7988,It's beautiful to see this young queen loving her CROWN 👑 . Black children can experience hair d...,beautiful see young queen loving crown black children experience hair discrimination early five ...
123468,Join us TODAY at 2pm ET for the NREI DC Community Conversation session sponsored by @GM! This #A...,join us today 2pm et nrei dc community conversation session sponsored gm alc51 session kicks fir...
148978,"The wealth-based legal system continues to exploit poor people, disproportionately Black &amp; B...",wealthbased legal system continues exploit poor people disproportionately black amp brown commun...


In [ ]:
# remove duplicates based on cleaned text column and check new length
data_filtered = data_filtered.drop_duplicates(
    subset=['text_cleaned']).reset_index(drop=True)

In [ ]:
final_tweets = data_filtered

final_tweets = final_tweets.rename(
    columns={'text': 'raw_text', 'text_cleaned': 'cleaned_text'})
final_tweets.to_excel(
    'drive/MyDrive/F24 Research/data/extended_cleaned_tweets.xlsx', index=False)

final_tweets = final_tweets[['raw_text', 'created_at', 'id', 'retweet_count', 'reply_count',
                             'like_count', 'quote_count', 'bookmark_count', 'impression_count',
                             'account_id', 'cleaned_text', 'identified_keywords']]
final_tweets.to_excel(
    'drive/MyDrive/F24 Research/data/simplified_cleaned_tweets.xlsx', index=False)

In [ ]:
historical_tweets = data_filtered[data_filtered['identified_keywords'].apply(
    len).apply(bool)].reset_index(drop=True)
# historical_tweets = historical_tweets[historical_tweets.apply(should_keep, axis=1)] # 8196 -> 8193

historical_tweets = historical_tweets.rename(
    columns={'text': 'raw_text', 'text_cleaned': 'cleaned_text'})
historical_tweets.to_excel(
    'drive/MyDrive/F24 Research/data/extended_historical_tweets.xlsx', index=False)

clean_historical_tweets = historical_tweets[['raw_text', 'created_at', 'id', 'retweet_count', 'reply_count',
                                             'like_count', 'quote_count', 'bookmark_count', 'impression_count',
                                             'account_id', 'cleaned_text', 'identified_keywords']]
clean_historical_tweets.to_excel(
    'drive/MyDrive/F24 Research/data/simplified_historical_tweets.xlsx', index=False)

- Create a 3rd subset of data that erases all the tweets with exclusions


In [ ]:
# parse in the excluded keywords and pair terms for 'voting rights act'
exclusions = keyword_df['Exclusions']
exclusions = exclusions.dropna()

voting_rights_pairs = ['1965', '1970', '1975', 'anniversary', 'years ago']

# if 'voting rights act' is in the keywords
# return true if any of the words in voting_rights_pairs are in the text
# return false if that is the only keyword
# return false if all other keywords are included in the ['voting rights act'] + exclusions

# if there is only one keyword
# return false if it is included in the exclusions or voting rights act

voting_and_exclusions = {'voting rights act'} | set(exclusions)
voting_and_exclusions = {
    keyword.lower() for keyword in voting_and_exclusions} | voting_and_exclusions

# Pre-compile regex for pair terms (to be used with voting rights act)
pair_pattern = re.compile(r'\b(' + '|'.join(re.escape(term)
                          for term in voting_rights_pairs) + r')\b', re.IGNORECASE)

# Define the function to decide if a row should be kept


def should_keep(row):
    # Keywords identified in the text
    keywords = set(row['identified_keywords'])

    # Check if 'voting rights act' is one of the keywords
    if 'voting rights act' in keywords:
        # Check if any pair terms are present in the raw text
        if bool(pair_pattern.search(row['raw_text'])):
            return True  # Keep the row if any pair terms are found

        # Return False if the only keyword is 'voting rights act' and no pair terms are found
        if len(keywords) == 1:
            return False

    # If all keywords are in exclusions, return False
    if keywords.issubset(voting_and_exclusions):
        return False

    # Otherwise, keep the row
    return True


data = historical_tweets.copy()
# Apply the function to the DataFrame
data = data[data.apply(should_keep, axis=1)]

In [ ]:
pattern = re.compile(r'\b(' + '|'.join(re.escape(ex)
                     for ex in voting_and_exclusions) + r')\b', re.IGNORECASE)

data = data[data['raw_text'].apply(lambda x: not bool(pattern.search(x)))]

In [ ]:
historical_tweets_without_exclusions = data

# historical_tweets = historical_tweets[historical_tweets.apply(should_keep, axis=1)] # 8196 -> 8193

historical_tweets_without_exclusions = historical_tweets_without_exclusions.rename(
    columns={'text': 'raw_text', 'text_cleaned': 'cleaned_text'})
historical_tweets_without_exclusions.to_excel(
    'drive/MyDrive/F24 Research/data/extended_historical_tweets_without_exclusions.xlsx', index=False)

clean_historical_tweets_without_exclusions = historical_tweets_without_exclusions[['raw_text', 'created_at', 'id', 'retweet_count', 'reply_count',
                                                                                   'like_count', 'quote_count', 'bookmark_count', 'impression_count',
                                                                                   'account_id', 'cleaned_text', 'identified_keywords']]
clean_historical_tweets_without_exclusions.to_excel(
    'drive/MyDrive/F24 Research/data/simplified_historical_tweets_without_exclusions.xlsx', index=False)


In [ ]:
df1 = historical_tweets
df2 = historical_tweets_without_exclusions

mask = df1.apply(tuple, axis=1).isin(df2.apply(tuple, axis=1))

indices_not_in_df2 = df1.index[~mask].tolist()

historical_tweets_only_exclusions = df1.iloc[indices_not_in_df2]


# historical_tweets = historical_tweets[historical_tweets.apply(should_keep, axis=1)] # 8196 -> 8193

historical_tweets_only_exclusions = historical_tweets_only_exclusions.rename(
    columns={'text': 'raw_text', 'text_cleaned': 'cleaned_text'})
historical_tweets_only_exclusions.to_excel(
    'drive/MyDrive/F24 Research/data/extended_historical_tweets_only_exclusions.xlsx', index=False)

clean_historical_tweets_only_exclusions = historical_tweets_only_exclusions[['raw_text', 'created_at', 'id', 'retweet_count', 'reply_count',
                                                                             'like_count', 'quote_count', 'bookmark_count', 'impression_count',
                                                                             'account_id', 'cleaned_text', 'identified_keywords']]
clean_historical_tweets_only_exclusions.to_excel(
    'drive/MyDrive/F24 Research/data/simplified_historical_tweets_only_exclusions.xlsx', index=False)


- Basic descriptives (total number of tweets, total number of Historical tweets, number of most frequent keywords)
- Don't include time wordings as keywords in the graph (on this day on)


In [ ]:
from collections import Counter
import ast

# Flatten the list of lists into a single list of keywords
all_keywords = [keyword for sublist in historical_tweets['identified_keywords']
                for keyword in sublist]

# Count the occurrences of each keyword
keyword_counts = Counter(all_keywords)

# Convert the counts to a pandas Series and sort it by count in descending order
keyword_series = pd.Series(keyword_counts).sort_values(ascending=False)

# Convert keywords in indices back to original format
keyword_series.index = [{k.lower(): k for k in keywords}[idx]
                        for idx in keyword_series.index]

# Filter out indices containing "day" or "years"
indices_to_drop = [
    idx for idx in keyword_series.index
    if 'day' in idx or 'years' in idx or re.search(r'\b\d{4}\b', idx)
]

# Drop the identified indices from the Series
keyword_series_cleaned = keyword_series.drop(indices_to_drop)


keyword_series_cleaned.name = 'Frequency'
keyword_series_cleaned.index.name = 'Keyword'
keyword_series_cleaned.to_excel(
    'drive/MyDrive/F24 Research/data/keyword_series.xlsx')
pd.read_excel('drive/MyDrive/F24 Research/data/keyword_series.xlsx')

In [ ]:
descriptives = pd.Series({'Total Number of Tweets': len(final_tweets),
                          'Total Number of Historical Tweets': len(historical_tweets),
                          'Total Number of Historical Tweets minus Exclusions': len(historical_tweets_without_exclusions)})
descriptives.name = 'Count'
descriptives.index.name = 'Description'
descriptives.to_excel('drive/MyDrive/F24 Research/data/descriptives.xlsx')

- Proportion of historical tweets or - as an alternative; a plot that has both a line/graph showing the number of total tweets and a line that shows the number of historical tweets


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the datasets
final_tweets = pd.read_excel(
    'drive/MyDrive/F24 Research/data/simplified_cleaned_tweets.xlsx')
historical_tweets = pd.read_excel(
    'drive/MyDrive/F24 Research/data/simplified_historical_tweets_no_exclusions.xlsx')

# Convert the 'created_at' column to datetime
final_tweets['created_at'] = pd.to_datetime(final_tweets['created_at'])
historical_tweets['created_at'] = pd.to_datetime(
    historical_tweets['created_at'])

# Group by date and count the number of tweets
final_tweets_counts = final_tweets.groupby(
    final_tweets['created_at'].dt.date).size()
historical_tweets_counts = historical_tweets.groupby(
    historical_tweets['created_at'].dt.date).size()

# Create a plot
plt.figure(figsize=(12, 6))

# Plot total tweets with filled area
plt.plot(final_tweets_counts.index, final_tweets_counts.values,
         label='Total Tweets', color='blue')
# plt.fill_between(final_tweets_counts.index, final_tweets_counts.values, color='blue')

# Plot historical tweets with filled area
plt.plot(historical_tweets_counts.index, historical_tweets_counts.values,
         label='Historical Tweets', color='orange')
# plt.fill_between(historical_tweets_counts.index, historical_tweets_counts.values, color='orange')

# Add labels and title
plt.title('Number of Tweets Over Time')
plt.xlabel('Date')
plt.ylabel('Number of Tweets')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.legend()
plt.tight_layout()  # Adjust layout to fit labels
plt.grid()

# Show the plot
plt.show()

- Frequencies of keywords over time


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the datasets
final_tweets = pd.read_excel(
    'drive/MyDrive/F24 Research/data/simplified_cleaned_tweets.xlsx')
historical_tweets = pd.read_excel(
    'drive/MyDrive/F24 Research/data/simplified_historical_tweets_no_exclusions.xlsx')

# Convert the 'identified_keywords' column to a list of lists
historical_tweets['identified_keywords'] = historical_tweets['identified_keywords'].apply(
    ast.literal_eval)

# Convert the 'created_at' column to datetime
final_tweets['created_at'] = pd.to_datetime(final_tweets['created_at'])
historical_tweets['created_at'] = pd.to_datetime(
    historical_tweets['created_at'])

# Load the CSV file
all_keywords_df = pd.read_excel(
    'drive/MyDrive/F24 Research/data/keyword_series.xlsx')

# Create a Series using column 0 as the index and column 1 as the values
keyword_series = pd.Series(
    all_keywords_df.iloc[:, 1].values, index=all_keywords_df.iloc[:, 0]).sort_values(ascending=False)

# Get the top 10 most popular keywords
top_keywords = keyword_series.index.tolist()[:10]

# top_keywords = keyword_series.index.tolist()[4:4+10]

# Create a DataFrame to count occurrences of top keywords over time
# Initialize a DataFrame to hold the counts
keyword_time_counts = pd.DataFrame()

# Loop through each of the top keywords and count occurrences over time
for keyword in top_keywords:
    keyword_counts_by_date = (
        historical_tweets[historical_tweets['identified_keywords'].apply(
            lambda x: keyword in x)]
        .groupby(historical_tweets['created_at'].dt.date)  # Group by date
        .size()  # Count occurrences
    )
    keyword_time_counts[keyword] = keyword_counts_by_date

# Fill NaN values with 0 (in case a keyword doesn't appear on a certain date)
keyword_time_counts = keyword_time_counts.fillna(0)

# Create a line plot for the frequency of the top keywords over time
plt.figure(figsize=(12, 6))

for keyword in top_keywords:
    plt.plot(keyword_time_counts.index,
             keyword_time_counts[keyword], label=keyword)

# Add labels and title
plt.title('Frequency of Top 10 Most Popular Keywords Over Time')
plt.xlabel('Date')
plt.ylabel('Frequency')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.legend(title='Keywords')
plt.tight_layout()  # Adjust layout to fit labels
plt.grid()

# Show the plot
plt.show()